<a href="https://colab.research.google.com/github/VVS1997/Crime-Data-Analytics-And-Prediction-/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

In [20]:
spark=SparkSession.builder.appName('Project_crime').getOrCreate()

In [21]:
spark

## Reading the data

In [ ]:
df1 = spark.read.csv("/content/Chicago_Crimes_2008_to_2011.csv", header=True, inferSchema=True)
df2 = spark.read.csv("/content/Chicago_Crimes_2012_to_2017.csv", header=True, inferSchema=True)


In [ ]:
df = df1.union(df2)


In [ ]:
df.write.csv("Crime_Dataset_concat.csv", header=True)


In [ ]:
df.show()


In [15]:
df.summary()


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [16]:
df.describe()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

## Looking at the data

In [ ]:
df.printSchema()


In [ ]:
df.count()

In [ ]:
num_rows1 = df.count()
num_cols1 = len(df.columns)
print("No. of Rows in application_record data frame:")
print(num_rows1)

print("No. of Columns in application_record data frame:")
print(num_cols1)

In [ ]:
from pyspark.sql.functions import col,sum


In [ ]:
null_counts = df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns))

In [ ]:
clean_df = df.dropna()

In [ ]:
null_counts2 = clean_df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in clean_df.columns))

In [ ]:
null_counts2.show()

In [ ]:
num_rows3 = clean_df.count()
num_cols3 = len(clean_df.columns)

In [ ]:
print("No. of Rows in Final data frame:")
print(num_rows3)

print("No. of Columns in Final data frame:")
print(num_cols3)

In [ ]:
  quartiles = clean_df.approxQuantile("X Coordinate", [0.25, 0.75], 0.01)
  iqr = quartiles[1] - quartiles[0]
  lower_bound = quartiles[0] - 1.5 * iqr
upper_bound = quartiles[1] + 1.5 * iqr
outliers = clean_df.filter((df["X Coordinate"] < lower_bound) | (df["X Coordinate"] > upper_bound))

In [ ]:
outliers.show()

In [ ]:
outliers.count()

In [ ]:
cleaned_df = clean_df.subtract(outliers)


In [ ]:
cleaned_df.count()


In [ ]:
 quartiles1 = cleaned_df.approxQuantile("Y Coordinate", [0.25, 0.75], 0.01)
 iqr = quartiles[1] - quartiles[0]
 lower_bound = quartiles[0] - 1.5 * iqr
 upper_bound = quartiles[1] + 1.5 * iqr
 outliers1 = cleaned_df.filter((df["Y Coordinate"] < lower_bound) | (df["Y Coordinate"] > upper_bound))

In [ ]:
outliers1.count()


In [ ]:
cleaned_df.count()


## Overview

In [ ]:
crime_type_groups = df.groupBy('primary_type').count()


In [ ]:
crime_type_counts = crime_type_groups.orderBy('count', ascending=False)


In [ ]:
crime_type_counts.show(truncate=False)

In [ ]:
counts_pddf = pd.DataFrame(crime_type_counts.rdd.map(lambda l: l.asDict()).collect())

In [ ]:
counts_pddf.head(10)

# Recorded Date

In [ ]:
from pyspark.sql.functions import *
import datetime


In [ ]:
df.select(min('date').alias('first_record_date'), max('date').alias('latest_record_date')).show(truncate=False)

In [ ]:
df = df.withColumn('date_time', to_timestamp('date', 'MM/dd/yyyy hh:mm:ss a'))\
       .withColumn('month', trunc('date_time', 'YYYY'))

In [ ]:
df.select(['date','date_time', 'month'])\
  .show(n=2, truncate=False)

In [ ]:
import datetime

In [ ]:
datetime.datetime.now()
datetime.datetime.strftime(datetime.datetime.now(), '%H')

 What time of the day are ciminal the busiest?

In [ ]:
# Extract the "hour" field from the date into a separate column called "hour"
df_hour = df.withColumn('hour', hour(df['date_time']))

In [ ]:
hourly_count = df_hour.groupBy(['primary_type', 'hour']).count().cache()
hourly_total_count = hourly_count.groupBy('hour').sum('count')

In [ ]:
hourly_count_pddf = pd.DataFrame(hourly_total_count.select(hourly_total_count['hour'], hourly_total_count['sum(count)'].alias('count'))\
                                .rdd.map(lambda l: l.asDict())\
                                 .collect())

In [ ]:
hourly_count_pddf = hourly_count_pddf.sort_values(by='hour')

In [ ]:
fig, ax = plt.subplots()
ax.plot(hourly_count_pddf['hour'], hourly_count_pddf['count'], label='Hourly Count')

ax.set(xlabel='Hour of Day', ylabel='Total records',
       title='Overall hourly crime numbers')
ax.grid(b=True, which='both', axis='y')
ax.legend()

### A closer look at crime date and time


In [ ]:
df_dates = df_hour.withColumn('week_day', dayofweek(df_hour['date_time']))\
                 .withColumn('year_month', month(df_hour['date_time']))\
                 .withColumn('month_day', dayofmonth(df_hour['date_time']))\
                 .withColumn('date_number', datediff(df['date_time'], to_date(lit('2001-01-01'), format='yyyy-MM-dd')))\
                 .cache()

In [ ]:
df_dates.select(['date', 'month', 'hour', 'week_day', 'year', 'year_month', 'month_day', 'date_number']).show(20, truncate=False)

## Day of week crime levels

In [ ]:
week_day_crime_counts = df_dates.groupBy('week_day').count()

In [ ]:
week_day_crime_counts_pddf = pd.DataFrame(week_day_crime_counts.orderBy('week_day').rdd.map(lambda e: e.asDict()).collect())

In [ ]:
sns.barplot(data=week_day_crime_counts_pddf, x='week_day', y='count')

# Month of year

In [ ]:
year_month_crime_counts = df_dates.groupBy('year_month').count()

In [ ]:
year_month_crime_counts_pddf = pd.DataFrame(year_month_crime_counts.orderBy('year_month').rdd.map(lambda e: e.asDict()).collect())

In [ ]:
year_month_crime_counts_pddf

In [ ]:
sns.barplot(data=year_month_crime_counts_pddf, y='count', x='year_month')

What happens in MAY through AUGUST?

 * Salary increase season?
 * Weather?

## we try to predict the primary type of crime?

In [ ]:
selected_features = [
 'location_description',
 'arrest',
 'domestic',
 'beat',
 'district',
 'ward',
 'community_area',
 'fbi_code',
 'hour',
 'week_day',
 'year_month',
 'month_day',
 'date_number']

In [ ]:
#schema of these selected features:

features_df = df_dates.select(selected_features)
features_df.printSchema()